In [1]:
#!pip install loguru
#!pip install pytz

In [2]:
import calendar_service

In [3]:
calendarService = calendar_service.CalendarService()

In [4]:
print(type(calendarService.timezone))
calendarService.timezone

<class 'pytz.UTC'>


<UTC>

In [5]:
calendarService.get_current_datetime()

'2025-03-30T21:04:12.967682+00:00'

In [6]:
calendarService.set_timezone("Europe/Istanbul")

2025-03-31 00:04:12.992 | INFO     | calendar_service:set_timezone:48 - Timezone updated to Europe/Istanbul


In [7]:
calendarService.get_current_datetime()

'2025-03-31T00:04:13.000885+03:00'

In [8]:
calendarService.get_events()

"Event 'Progress meeting' from 2025-03-24T17:00:00+03:00 to 2025-03-24T18:00:00+03:00\nEvent 'Progress meeting' from 2025-03-26T14:00:00+03:00 to 2025-03-26T15:00:00+03:00\nEvent 'Test Meeting' from 2025-03-27T01:19:52+03:00 to 2025-03-27T02:19:52+03:00\nEvent 'Progress meeting' from 2025-03-28T14:00:00+03:00 to 2025-03-28T15:00:00+03:00"

In [12]:
calendarService.get_free_availability(
    time_min="2025-03-31T00:00:00",
    time_max="2025-03-31T23:59:59"
)


2025-03-31 00:06:26.724 | DEBUG    | calendar_service:get_free_availability:150 - Querying between 2025-03-30T21:00:00Z and 2025-03-31T20:59:59Z (converted from local timezone)


'Free from 2025-03-31T00:00:00+03:00 to 2025-03-31T14:00:00+03:00\nFree from 2025-03-31T15:00:00+03:00 to 2025-03-31T15:30:00+03:00\nFree from 2025-03-31T16:15:00+03:00 to 2025-03-31T16:30:00+03:00\nFree from 2025-03-31T17:15:00+03:00 to 2025-03-31T23:59:59+03:00'

In [11]:
calendarService.create_event(
    summary="Team Sync",
    start_time="2025-03-31T16:30:00",
    duration_minutes=45,
    description="Weekly check-in",
    attendees=["amasya485@gmail.com"]
)


2025-03-31 00:05:04.863 | INFO     | calendar_service:create_event:127 - Created event 'Team Sync' at 2025-03-31T16:30:00+03:00 lasting 45 minutes. Link: https://www.google.com/calendar/event?eid=bWRuMmNuMzZydG9ubGluNTFhaXNraXRmaTggZWdlQG5hbm9taW5kcy5haQ


"Created event 'Team Sync' at 2025-03-31T16:30:00+03:00 lasting 45 minutes. Link: https://www.google.com/calendar/event?eid=bWRuMmNuMzZydG9ubGluNTFhaXNraXRmaTggZWdlQG5hbm9taW5kcy5haQ"